# Generating Lookup Tables for Geolocation
This notebook details the steps to generate the lookup tables for geolocation.

In [315]:
import pandas as pd
pd.set_option('display.max_rows', 300)

# International Cities and Counties
World cities with populations >100K, Canadian cities, Canadian counties, UK cities

In [377]:
# Only need first column of data
un_cities = pd.read_csv('../public_data/uncities.csv').iloc[:,0].dropna()
un_cities.head()

0    Population des capitales et des villes de 100 ...
1    Continent, country or area, date, code and cit...
5                                     AFRICA - AFRIQUE
6                                    Algeria - Algérie
7                           16 IV 2008 (CDJC)         
Name: 8. Population of capital cities and cities of 100 000 or more inhabitants: latest available year,  2000 - 2019, dtype: object

In [367]:
uk_areas = pd.read_csv('../public_data/ukcensusareas.csv', skiprows=1, 
                        usecols=['Unnamed: 2']).set_axis(['area_name'], axis=1).iloc[:,0]
uk_areas.head()

0             England
1    Northern Ireland
2            Scotland
3               Wales
4          North East
Name: area_name, dtype: object

In [361]:
load_cols = ['GEO_NAME', 'DIM: Profile of Census Metropolitan Areas/Census Agglomerations (2247)',
            'Dim: Sex (3): Member ID: [1]: Total - Sex']
canada_areas = pd.read_csv('../public_data/canadacensusareas.csv', low_memory=False, usecols=load_cols)\
                  .set_axis(['area_name', 'variable', 'population'], axis=1)
canada_areas = canada_areas[canada_areas['variable'] == 'Population, 2016'].iloc[:,0]
canada_areas.head()

0                St. John's
2247            Bay Roberts
4494    Grand Falls-Windsor
6741                 Gander
8988           Corner Brook
Name: area_name, dtype: object

In [359]:
un_countries = pd.read_csv('../public_data/uncountries.csv', header=None).iloc[:,0]
un_countries.head()

0       Afghanistan
1     Åland Islands
2           Albania
3           Algeria
4    American Samoa
Name: 0, dtype: object

In [393]:
foreign_entities = set(un_cities).union(uk_areas).union(canada_areas).union(un_countries)
foreign_entities = {s.lower() for s in foreign_entities}

# US City Data

In [519]:
states = pd.read_csv('../public_data/usstateabbreviations.csv', names=['abb', 'name'])['name']

# Filter by population
us_cities_df = us_cities_df[us_cities_df['population'] >= 5000]

def extract_state(state):
    split = state.split(',')[-1].strip()
    # Some state strings have multiple punctuation and require extra filtering,
    # e.g., "Lynchburg, Moore county metropolitan government, Tennessee"
    if ';' in split:
        return split.split(';')[-1].strip()
    else:
        return split

In [522]:
import re
def tokenize_cityname(row):
    state = row['state']
    city = row['city_name']
    city = city.split(',')[0]     # Remove State from city name
    # These strings were appended by the census; 'and' is for multiple area labels
    area_names = ['city', 'village', 'borough', 'town', 'CDP', 'municipality', 'and']
    city_and_state = city[:]
    for area_name in area_names:
        city_and_state = city_and_state.replace(' '+area_name, '', 1)
    stripped = city_and_state.strip(' ,')
    # Ensures strings like "Arizona City, Arizona" are changed to "Arizona City"
    # and not "City, Arizona" after strip.
    return state + ' City' if state in stripped else stripped

In [462]:
len(us_cities_df['state'].unique())

52

In [521]:

us_cities_df = pd.read_csv('../public_data/usacscities.csv', skiprows=1,
                          usecols=['Geographic Area Name', 'Estimate!!Total'])\
                           .set_axis(['city_name', 'population'], axis=1)
us_cities_df['state'] = us_cities_df['city_name'].apply(extract_state)
us_cities_df['city_name_formatted'] = us_cities_df.apply(tokenize_cityname, axis=1)
us_cities_df = us_cities_df.sort_values('population', ascending=False)#.duplicated(subset=['city_name_formatted'], keep='first')
us_cities_df = us_cities_df.drop_duplicates(subset=['city_name_formatted'], keep='first')
us_cities_df.head()
us_cities_df[us_cities_df['city_name_formatted'].str.contains('eorge')]

,city_name,population,state,city_name_formatted
17551,"New York city, New York",8426743,New York,New York City
2725,"Los Angeles city, California",3900794,California,Los Angeles
6283,"Chicago city, Illinois",2717534,Illinois,Chicago
25203,"Houston city, Texas",2217706,Texas,Houston
22711,"Philadelphia city, Pennsylvania",1555072,Pennsylvania,Philadelphia


## Format the City Name from the list of cities to include only the city name

In [230]:
re.split('\W+', 'St. George')

['St', 'George']

In [495]:
us_cities_df.head()
us_cities_df = us_cities_df.sort_values('population').duplicated(subset=['city_name_formatted'], keep='first')

ValueError: No axis named population for object type <class 'pandas.core.series.Series'>

In [489]:
us_cities_df.head()

28841    False
15964    False
12520    False
18587    False
4895     False
dtype: bool

In [494]:
print(len(us_cities_df[['state', 'city_name_formatted']].drop_duplicates()))
us_cities_df[us_cities_df.duplicated(subset=['state', 'city_name_formatted'])]

KeyError: "None of [Index(['state', 'city_name_formatted'], dtype='object')] are in the [index]"

So only 3 townships were dropped.

## Add population data

In [246]:
us_pops = pd.read_csv('../public_data/uscitypops.csv', usecols=['city', 'state_name', 'population'])
print(len(us_pops))
us_pops.head()

28338


,city,state_name,population
0,New York,New York,18713220
1,Los Angeles,California,12750807
2,Chicago,Illinois,8604203
3,Miami,Florida,6445545
4,Dallas,Texas,5743938


In [313]:
us_pops[us_pops['city']=='Darien']

,city,state_name,population
2220,Darien,Illinois,21628
8072,Darien,Georgia,3580
12030,Darien,Wisconsin,1590


In [280]:
us_pops[us_pops.duplicated(subset=['city', 'state_name'])]

,city,state_name,population
4553,Woodbury,New York,8852
5301,Middletown,Pennsylvania,7212
8018,Oakwood,Ohio,3624
8908,Midway,Florida,3000
11916,San Antonio,Puerto Rico,1625
12023,Midway,Florida,1592
12633,Oakland,Pennsylvania,1422
12648,San Antonio,Puerto Rico,1419
13691,Georgetown,Pennsylvania,1182
15813,Chula Vista,Texas,829


In [308]:
city_state_pops = pd.merge(us_cities_df.rename(columns={'state':'state_name', 'city_name_formatted':'city'})[['state_name', 'city']],
                           us_pops, 
                           how='left', 
                           on=['city', 'state_name'])\
                    .sort_values('population', ascending=False)\
                    .reset_index()

In [309]:
city_state_pops.head()

,index,state_name,city,population
0,537,California,Los Angeles,12750807.0
1,1658,Illinois,Chicago,8604203.0
2,1197,Florida,Miami,6445545.0
3,4735,Texas,Dallas,5743938.0
4,4401,Pennsylvania,Philadelphia,5649300.0


In [311]:
len(city_state_pops[city_state_pops['population'].isna()].head(300))

194

In [310]:
city_state_pops[city_state_pops['population'].isna()].head(300)

,index,state_name,city,population
5378,105,Alaska,Lakes,NaN
5379,187,Arizona,Village of Oak Creek (Big Park),NaN
5380,299,California,Bonadelle Ranchos-Madera Ranchos,NaN
5381,402,California,El Paso de Robles (Paso Robles),NaN
5382,405,California,El Sobrante (Riverside County),NaN
5383,695,California,San Buenaventura (Ventura),NaN
5384,754,California,Spring Valley (San Diego County),NaN
5385,916,Colorado,Twin Lakes (Adams County),NaN
5386,924,Connecticut,Bethel,NaN
5387,930,Connecticut,Darien,NaN
